In [18]:
import torch
from torchvision.datasets import MNIST
from torch import nn
from torch.nn import functional as F
from torchvision.transforms import ToTensor
import numpy as np

In [2]:
dataset = MNIST("./data", transform=ToTensor())

In [3]:
from torch.utils.data import DataLoader

In [4]:
data_loader = DataLoader(dataset,batch_size=4)

In [22]:
import matplotlib.pyplot as plt
print(dataset.__getitem__(2)[0].size())


torch.Size([1, 28, 28])


In [5]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(10,10)
        self.fc2 = nn.Linear(10, 784)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = x.view(28, 28)
        return x

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # output of conv1 is (10, 24, 24)
        self.conv1 = nn.Conv2d(1, 10, 5)
        # input of conv2 will be (10, 12, 12)
        # output of conv2 will be (20, 8, 8)
        self.conv2 = nn.Conv2d(10, 20, 5)
        #TODO: figure out dimension
        
        self.fc1 = nn.Linear(20 * 4 * 4, 60)
        self.fc2 = nn.Linear(60, 20)
        self.fc3 = nn.Linear(20, 2)
    
    def forward(self, x):
        
        # output shape=(10, 12, 12)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # output shape = (20, 4, 4)
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        # reshape to 20 * 4 * 4
        x = x.view(-1, self.num_flat_features(x))
        # output shape=
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features    

In [8]:
discriminator = Discriminator()
print(len(list(discriminator.parameters())))
#discriminator.forward(dataset.__getitem__(2)[0])

10


In [16]:
discriminator(dataset.__getitem__(2)[0].unsqueeze(0)).size()

torch.Size([1, 2])

In [11]:
criterion = nn.CrossEntropyLoss()

In [26]:
loss = criterion(discriminator(dataset.__getitem__(2)[0].unsqueeze(0)), torch.tensor([1],dtype=torch.int8).unsqueeze(0))

RuntimeError: Expected object of scalar type Long but got scalar type Char for argument #2 'target'

In [22]:
torch.Tensor(np.array([1],dtype="long"))

tensor([1.])